<a href="https://colab.research.google.com/github/yasserrida/active-learning-based-on-volumes/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>